# Clustering ensemble

Grupowanie może być także używane jako preprocessing do klasyfikacji. Idea jest zbliżona do komitetów klasyfikatorów.

ZADANIE: Proszę wziąć dane ecoli z UCI. Podzielić na zbiór testowy i treningowy.

1. Uczymy logistyczną regresję i testujemy (accuracy)

2. Na zbiorze treningowym robimy k-means (5 klastrów) co daje podział. W każdym klastrze uczymy logistyczną regresję. Testujemy przypisując punkt do najbliższego klastra, a następnie przewidując modelem klasyfikacyjnym z tego klastra.

Które podejście daje lepszy wynik. Czy drugie podejście radzi sobie z klasami nieseparowalnymi liniowo?

In [47]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    filepath_or_buffer='data_cl/ecoli.dat', 
    header=None, 
    sep=',')

X = np.array(df.iloc[:, :-1])
X
print(X.shape)
y = df.iloc[:,-1]

labels = set(y)
labels
#{cp,im,imS,imL,imU,om,omL,pp}

(336, 7)


{'cp', 'im', 'imL', 'imS', 'imU', 'om', 'omL', 'pp'}

In [48]:
z = {'cp':0, 'im':1, 'imL':2, 'imS':3, 'imU':4, 'om':5, 'omL':6, 'pp':7}
z

{'cp': 0, 'im': 1, 'imL': 2, 'imS': 3, 'imU': 4, 'om': 5, 'omL': 6, 'pp': 7}

In [49]:
Y = np.array([z[key] for key in y])
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

In [50]:
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)
print(X_train.shape[0])
print(X_test.shape[0])

268
68


In [1]:
n = 5

from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=n, random_state = 42)
# slice matrix so we only include the 0/1 indicator columns in the clustering
clust_train = cluster.fit_predict(X_train)

clust_test = cluster.predict(X_test)

NameError: name 'X_train' is not defined

In [52]:
#wyciagam dane z kolejnych klas


def split(X, y, clustering):
    data = []
    labels = []
    for j in range(len(set(clustering))):
        data.append([X[i] for i in range(X.shape[0]) if clustering[i] == j])
        labels.append([y[i] for i in range(y.shape[0]) if clustering[i] == j])
   
    return(data, labels)     


In [53]:
#wyciagam dane z kolejnych klas


def split(X, y, clustering):
    data = []
    labels = []
    for j in range(len(set(clustering))):
        data.append(X[clustering == j])
        labels.append(y[clustering == j])
#         labels.append([y[i] for i in range(y.shape[0]) if clustering[i] == j])
   
    return(data, labels)     


In [54]:
data_train, labels_train = split(X_train, y_train, clust_train)
data_test, labels_test = split(X_test, y_test, clust_test)

In [55]:
data_train


[array([[24., 57., 48.,  5., 63., 34., 43.],
        [22., 34., 48.,  5., 42., 29., 39.],
        [33., 47., 48.,  5., 53., 18., 29.],
        [35., 34., 48.,  5., 46.,  3., 27.],
        [26.,  4., 48.,  5., 36., 26., 37.],
        [61., 36., 48.,  5., 49., 35., 44.],
        [58., 55., 48.,  5., 57.,  7., 74.],
        [38.,  4., 48.,  5., 63., 25., 35.],
        [38., 46., 48.,  5., 48., 22., 29.],
        [ 4., 45., 48.,  5., 38., 22.,  0.],
        [61.,  6., 48.,  5., 44., 39., 38.],
        [23., 34., 48.,  5., 43., 26., 37.],
        [34., 45., 48.,  5., 38., 24., 35.],
        [ 4.,  5., 48.,  5., 45., 39., 47.],
        [44., 51., 48.,  5., 47., 26., 36.],
        [35., 48., 48.,  5., 56.,  4., 48.],
        [29., 39., 48.,  5., 52.,  4., 48.],
        [34., 28., 48.,  5., 41., 35., 44.],
        [38.,  3., 48.,  5., 43., 29., 39.],
        [ 4., 31., 48.,  5., 41., 29., 39.],
        [37., 44., 48.,  5., 42., 39., 47.],
        [27.,  3., 48.,  5., 71., 28., 39.],
        [4

In [56]:
from sklearn.metrics import accuracy_score

logreg = linear_model.LogisticRegression(C=1e5, random_state= 42) 
acc_te = 0
acc_tr = 0
for i in range(len(set(clust_train))):
    logreg.fit(data_train[i], labels_train[i])
    y_train_pred = logreg.predict(data_train[i])
    acc_train = accuracy_score(labels_train[i], y_train_pred)
    print("Dokładność na zbiorze treningowym ", acc_train)
    acc_tr = acc_tr + len(labels_train[i])*acc_train
    
    y_test_pred = logreg.predict(data_test[i])
    acc_test = accuracy_score(labels_test[i], y_test_pred)
    print("Dokładność na zbiorze treningowym ", acc_test)
    print("-------------------------------")
    acc_te = acc_te + len(labels_test[i])*acc_test
    
acc_tr = acc_tr/len(X_train)
acc_te = acc_te/len(X_test)

print(acc_tr)
print(acc_te)


Dokładność na zbiorze treningowym  0.9223300970873787
Dokładność na zbiorze treningowym  1.0
-------------------------------
Dokładność na zbiorze treningowym  0.717391304347826
Dokładność na zbiorze treningowym  0.6666666666666666
-------------------------------
Dokładność na zbiorze treningowym  0.9122807017543859
Dokładność na zbiorze treningowym  0.8
-------------------------------
Dokładność na zbiorze treningowym  1.0
Dokładność na zbiorze treningowym  1.0
-------------------------------
Dokładność na zbiorze treningowym  1.0
Dokładność na zbiorze treningowym  0.7777777777777778
-------------------------------
0.9029850746268657
0.8823529411764706


In [30]:
logreg = linear_model.LogisticRegression(C=1e5, random_state = 42) # w prakcy powinniśmy dopasować parametr c
logreg.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [31]:
from sklearn.metrics import accuracy_score

y_train_pred = logreg.predict(X_train)
print("Dokładność na zbiorze treningowym ", accuracy_score(y_train, y_train_pred))



y_test_pred = logreg.predict(X_test)
print("Dokładność na zbiorze test ", accuracy_score(y_test, y_test_pred))

Dokładność na zbiorze treningowym  0.7985074626865671
Dokładność na zbiorze test  0.8235294117647058
